In [40]:
import pandas as pd
import numpy as np
from scipy.stats import poisson


file_path = 'data/529_skyscrapers/people_by_minute.csv'
data = pd.read_csv(file_path)


start_time = pd.to_datetime('12:00', format='%H:%M')
data['time'] = pd.to_datetime(data['time'], format='%H:%M')

service_rate = 8

data['queue_length'] = 0.0
current_queue = 0

for i, row in data.iterrows():
    arrivals = row['average_people']
    current_queue = max(current_queue + arrivals - service_rate, 0)
    data.at[i, 'queue_length'] = current_queue


max_queue_time = data.loc[data['queue_length'].idxmax(), 'time'].strftime('%H:%M')
data['queue_increase'] = data['queue_length'].shift(-15) - data['queue_length']
worst_delay_time = data.loc[data['queue_increase'].idxmax(), 'time'].strftime('%H:%M')



print(f"{max_queue_time},{worst_delay_time}")

14:10,12:50


In [ ]:
import numpy as np, pandas as pd, heapq, math, time, os, sys
np.random.seed(0)

df = pd.read_csv('data/529_skyscrapers/people_by_minute.csv')
minutes = df['time'].tolist()
lambdas = df['average_people'].values

NUM_MIN = len(lambdas)
TOTAL_AVG = lambdas.sum()

NUM_DAYS = 2000
SERVICE_TIME = 0.5
SERVERS_FULL = 4
SERVERS_AFTER_CLOSED = 3
MAX_WAIT_THRESHOLD = 10.0

counts = np.random.poisson(lam=lambdas, size=(NUM_DAYS, NUM_MIN))
total_events = counts.sum()
arrivals_by_day = []
for d in range(NUM_DAYS):
    idxs = np.nonzero(counts[d])[0]
    times = []
    for m in idxs:
        c = counts[d, m]
        times.append(m + np.random.random(size=c))
    if len(times) == 0:
        arrivals_by_day.append(np.array([]))
    else:
        arr = np.concatenate(times)
        arr.sort()
        arrivals_by_day.append(arr)

def simulate_day(arr_times, closure_minute=None):
    n = len(arr_times)
    if n == 0:
        return np.array([]), np.array([], dtype=int)
    start_times = np.empty(n, dtype=float)
    server_ids = np.empty(n, dtype=int)
    next_free = [0.0]*SERVERS_FULL
    closed_server = None
    closed_done = False
    heap = [(0.0, i) for i in range(SERVERS_FULL)]
    heapq.heapify(heap)
    i = 0
    while i < n:
        t = arr_times[i]
        if (closure_minute is not None) and (not closed_done) and (t >= closure_minute):
            heap_list = list(heap)
            max_server = max(heap_list, key=lambda x: x[0])[1]
            closed_server = max_server
            heap = [(nf,sid) for (nf,sid) in heap if sid != closed_server]
            heapq.heapify(heap)
            closed_done = True
        if not heap:
            earliest = min(next_free)
            server = next_free.index(earliest)
            start = max(t, earliest)
            start_times[i] = start
            server_ids[i] = server
            next_free[server] = start + SERVICE_TIME
            i += 1
            continue
        nf, sid = heapq.heappop(heap)
        nf = next_free[sid]
        start = max(t, nf)
        start_times[i] = start
        server_ids[i] = sid
        next_free[sid] = start + SERVICE_TIME
        heapq.heappush(heap, (next_free[sid], sid))
        i += 1
    return start_times, server_ids

sample_times = np.arange(NUM_MIN) + 0.5
queue_counts_accum = np.zeros(NUM_MIN, dtype=float)
wait_sum_per_min = np.zeros(NUM_MIN, dtype=float)
wait_count_per_min = np.zeros(NUM_MIN, dtype=int)

t0 = time.time()
for d in range(NUM_DAYS):
    arr = arrivals_by_day[d]
    start_times, server_ids = simulate_day(arr, closure_minute=None)
    if len(arr) == 0:
        continue
    waits = start_times - arr
    arr_min = np.floor(arr).astype(int)
    for m, w in zip(arr_min, waits):
        if 0 <= m < NUM_MIN:
            wait_sum_per_min[m] += w
            wait_count_per_min[m] += 1

    idx_arr = np.searchsorted(arr, sample_times, side='right')
    idx_start = np.searchsorted(start_times, sample_times, side='right')
    waiting_at_t = idx_arr - idx_start
    queue_counts_accum += waiting_at_t

avg_queue_per_min = queue_counts_accum / NUM_DAYS
avg_wait_per_min = np.divide(wait_sum_per_min, wait_count_per_min, out=np.zeros_like(wait_sum_per_min), where=wait_count_per_min>0)

idx_q1 = int(np.argmax(avg_queue_per_min))
time_q1 = minutes[idx_q1]

delta_wait = np.full(NUM_MIN, -np.inf)
for m in range(NUM_MIN-15):
    if wait_count_per_min[m] > 0 and wait_count_per_min[m+15] > 0:
        delta_wait[m] = avg_wait_per_min[m+15] - avg_wait_per_min[m]
idx_q2 = int(np.nanargmax(delta_wait))
time_q2 = minutes[idx_q2]

candidate_idxs = list(range(0, NUM_MIN, 10))

def check_closure_candidate(idx_min):
    closure_time = idx_min
    success_days = 0
    for d in range(NUM_DAYS):
        arr = arrivals_by_day[d]
        if len(arr) == 0:
            success_days += 1
            continue
        start_times, _ = simulate_day(arr, closure_minute=closure_time)
        mask = arr >= closure_time
        if not np.any(mask):
            success_days += 1
            continue
        waits_after = start_times[mask] - arr[mask]
        max_wait = waits_after.max()
        if max_wait <= MAX_WAIT_THRESHOLD:
            success_days += 1
    frac = success_days / NUM_DAYS
    return frac


lo = 0
hi = len(candidate_idxs)-1
best_idx = None
while lo <= hi:
    mid = (lo+hi)//2
    idx_min = candidate_idxs[mid]
    frac = check_closure_candidate(idx_min)
    if frac >= 0.9:
        best_idx = idx_min
        hi = mid-1
    else:
        lo = mid+1

time_q3 = minutes[best_idx]

def round_to_nearest_10(time_str):
    hh, mm = map(int, time_str.split(':'))
    total_min = hh*60 + mm
    rem = total_min % 10
    if rem < 5:
        rounded = total_min - rem
    else:
        rounded = total_min + (10-rem)
    rh = rounded//60
    rm = rounded%60
    return f"{rh:02d}:{rm:02d}"

ans1 = round_to_nearest_10(time_q1)
ans2 = round_to_nearest_10(time_q2)
ans3 = round_to_nearest_10(time_q3)

print("Final (rounded) answers:", ans1, ans2, ans3)